In [ ]:
#import sys
import numpy as np
from importlib import reload
import h5py
import os
import copy
import gc
import matplotlib.gridspec as gridspec
import matplotlib

from matplotlib import pyplot as plt, patches
from matplotlib import cm
from matplotlib.colors import Normalize, LogNorm 
from matplotlib.ticker import MultipleLocator, FuncFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
#import matplotlib as mpl

import astropy.io.fits as fits
import astropy.units as u
from astropy.wcs import WCS
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord, ICRS, Galactic

from scipy.interpolate import interpn
from scipy.optimize import curve_fit
from scipy.stats import linregress
from scipy import stats

from reproject import reproject_to_healpix, reproject_from_healpix
from reproject import reproject_interp
import healpy as hp

In [ ]:
def FD_moments(data,peakthresh,maxFD,RM_arr,M0=True,M1=True,M2=False,*args,**kwargs):
 
    dFD = abs(RM_arr[1]-RM_arr[0])

    # cut out FD range chosen:
    data_use = data[(abs(RM_arr) <= maxFD),:,:]    
    RM_arr_use = RM_arr[(abs(RM_arr) <= maxFD)]  

    # set any data points below PI threshold to NaN:
    data_use[data_use < peakthresh] = np.nan

    moments = {}
    if M0:
        M0_data = dFD*np.nansum(data_use,axis=0)
        M0_data[M0_data == 0] = np.nan
        moments['M0'] = M0_data
    if M1:
        M1_data = dFD*np.nansum(data_use*RM_arr_use[:,np.newaxis,np.newaxis],axis=0)/M0_data
        moments['M1'] = M1_data
    if M2:
        M2_data = np.sqrt(dFD*np.nansum(data_use*(RM_arr_use[:,np.newaxis,np.newaxis]-M1_data)**2,axis=0)/M0_data)
        moments['M2'] = M2_data
        
    return(moments)

## CHIME

In [ ]:
hdu_chime = fits.open('/srv/data/chime/chime_FD_Oct2023_586_729/FDF_clean_tot.fits')
#hdu_chime = fits.open('/srv/data/chime/chime_FD_May2024_400_729/FDF_clean_tot.fits')
hdr_chime = hdu_chime[0].header
chime_FD  = hdu_chime[0].data

FD_ax_chime = WCS(hdr_chime).all_pix2world(0,0,range(chime_FD.shape[0]),0)[2]
print(FD_ax_chime[0],FD_ax_chime[-1])
print(chime_FD.shape)

In [ ]:
plt.imshow(chime_FD[200],vmin=0,vmax=1,cmap='cubehelix',origin='lower')

## DRAGONS

In [ ]:
hdu_dragons = fits.open('/srv/data/dragons/Mar2024_500_1000_MHz_cube/FD_cube_DRAGONS_CLEAN_32.fits')
hdr_dragons = hdu_dragons[0].header
dragons_FD  = hdu_dragons[0].data

FD_ax_dragons = WCS(hdr_dragons).all_pix2world(0,0,range(dragons_FD.shape[0]),0)[2]
print(FD_ax_dragons[0],FD_ax_dragons[-1])
print(dragons_FD.shape)

In [ ]:
plt.imshow(dragons_FD[400],vmin=0,vmax=15,cmap='cubehelix',origin='lower')

## GMIMS-HBN

In [ ]:
hdu_hbn = fits.open('/srv/data/gmims/gmims-hbn/GMIMS-HBN_v1_gal_car_FD_PI.fits')
hdr_hbn = hdu_hbn[0].header
hbn_FD  = hdu_hbn[0].data

FD_ax_hbn = WCS(hdr_hbn).all_pix2world(0,0,range(hbn_FD.shape[0]),0)[2]
print(FD_ax_hbn[0],FD_ax_hbn[-1])
print(hbn_FD.shape)

In [ ]:
plt.imshow(hbn_FD[100],vmin=0,vmax=1,cmap='cubehelix',origin='lower')

## GMIMS-LBS

In [ ]:
hdu_lbs = fits.open('/srv/data/gmims/gmims-lbs/parkes_pi_gal.clean.fits')
hdr_lbs = hdu_lbs[0].header
lbs_FD  = hdu_lbs[0].data

FD_ax_lbs = WCS(hdr_lbs).all_pix2world(0,0,range(lbs_FD.shape[0]),0)[2]
print(FD_ax_lbs[0],FD_ax_lbs[-1])
print(lbs_FD.shape)

In [ ]:
plt.imshow(lbs_FD[200],vmin=0,vmax=2,cmap='cubehelix',origin='lower')

## STAPS

In [ ]:
hdu_staps = fits.open('/srv/data/gmims/gmims-hbs/Jan2024/P.cln.fits')
hdr_staps = hdu_staps[0].header
staps_FD  = hdu_staps[0].data

FD_ax_staps = WCS(hdr_staps).all_pix2world(0,0,range(staps_FD.shape[0]),0)[2]
print(FD_ax_staps[0],FD_ax_staps[-1])
print(staps_FD.shape)

In [ ]:
plt.imshow(staps_FD[50],vmin=0,vmax=0.2,cmap='cubehelix',origin='lower')

## Calculate all moments

In [ ]:
thresholding = False

if thresholding:
    #M_chime   = FD_moments(chime_FD,   0.08, 100, FD_ax_chime,  M0=True,M1=True,M2=False)
    #M_hbn     = FD_moments(hbn_FD,     0.04, 450, FD_ax_hbn,  M0=True,M1=True,M2=False)
    #M_dragons = FD_moments(dragons_FD,  0.5, 100, FD_ax_dragons,M0=True,M1=True,M2=False)
    #M_lbs     = FD_moments(lbs_FD,     0.04, 80,  FD_ax_lbs,  M0=True,M1=True,M2=False)
    M_staps   = FD_moments(staps_FD,   0.02, 450, FD_ax_staps,  M0=True,M1=True,M2=False)
else:
    print('not thresholding!')
    #M_chime   = FD_moments(chime_FD,   0.0, 100, FD_ax_chime,  M0=True,M1=True,M2=False)
    #M_hbn     = FD_moments(hbn_FD,     0.0, 450, FD_ax_hbn,  M0=True,M1=True,M2=False)
    #M_dragons = FD_moments(dragons_FD, 0.0, 100, FD_ax_dragons,M0=True,M1=True,M2=False)
    #M_lbs     = FD_moments(lbs_FD,     0.0, 80,  FD_ax_lbs,  M0=True,M1=True,M2=False)
    M_staps   = FD_moments(staps_FD,   0.0, 450, FD_ax_staps,  M0=True,M1=True,M2=False)

In [ ]:
#print(M_chime['M0'].shape)
#print(M_hbn['M0'].shape)
#print(M_dragons['M0'].shape)
#print(M_lbs['M0'].shape)
print(M_staps['M0'].shape)

## Everything to Healpix

In [ ]:
hdr2D_chime = hdr_chime.copy()
hdr2D_chime['NAXIS'] = 2
for card in hdr_chime.cards:
    try:
        if card[0][5] == '3':
            del hdr2D_chime[card[0]]
    except:
        pass
#print(repr(hdr2D_chime))

hdr2D_hbn = hdr_hbn.copy()
hdr2D_hbn['NAXIS'] = 2
hdr2D_hbn['WCSAXES'] = 2
for card in hdr_hbn.cards:
    try:
        if card[0][5] == '3':
            del hdr2D_hbn[card[0]]
    except:
        pass
#print(repr(hdr2D_gmims))

hdr2D_dragons = hdr_dragons.copy()
hdr2D_dragons['NAXIS'] = 2
hdr2D_dragons['WCSAXES'] = 2
for card in hdr_dragons.cards:
    try:
        if card[0][5] == '3':
            del hdr2D_dragons[card[0]]
    except:
        pass


hdr2D_lbs = hdr_lbs.copy()
hdr2D_lbs['NAXIS'] = 2
hdr2D_lbs['WCSAXES'] = 2
for card in hdr_lbs.cards:
    try:
        if card[0][5] == '3':
            del hdr2D_lbs[card[0]]
    except:
        pass


hdr2D_staps = hdr_staps.copy()
hdr2D_staps['NAXIS'] = 2
hdr2D_staps['WCSAXES'] = 2
for card in hdr_staps.cards:
    try:
        if card[0][5] == '3':
            del hdr2D_staps[card[0]]
    except:
        pass

M1_hpx_chime,   footprint = reproject_to_healpix((M_chime['M1'],   hdr2D_chime),'galactic', nside=512)
M1_hpx_hbn,     footprint = reproject_to_healpix((M_hbn['M1'],   hdr2D_hbn),'galactic', nside=512)
M1_hpx_dragons, footprint = reproject_to_healpix((M_dragons['M1'], hdr2D_dragons),'galactic', nside=512)
M1_hpx_lbs,     footprint = reproject_to_healpix((M_lbs['M1'], hdr2D_lbs),'galactic', nside=512)
M1_hpx_staps,   footprint = reproject_to_healpix((M_staps['M1'], hdr2D_staps),'ICRS', nside=512)

M0_hpx_chime,   footprint = reproject_to_healpix((M_chime['M0'],   hdr2D_chime),'galactic', nside=512)
M0_hpx_hbn,     footprint = reproject_to_healpix((M_hbn['M0'],   hdr2D_hbn),'galactic', nside=512)
M0_hpx_dragons, footprint = reproject_to_healpix((M_dragons['M0'], hdr2D_dragons),'galactic', nside=512)
M0_hpx_lbs,     footprint = reproject_to_healpix((M_lbs['M0'], hdr2D_lbs),'galactic', nside=512)
M0_hpx_staps,   footprint = reproject_to_healpix((M_staps['M0'], hdr2D_staps),'ICRS', nside=512)

In [ ]:
hdr2D_staps = hdr_staps.copy()
hdr2D_staps['NAXIS'] = 2
hdr2D_staps['WCSAXES'] = 2
for card in hdr_staps.cards:
    try:
        if card[0][5] == '3':
            del hdr2D_staps[card[0]]
    except:
        pass

M1_hpx_staps,   footprint = reproject_to_healpix((M_staps['M1'], hdr2D_staps),'ICRS', nside=512)

In [ ]:
M0_hpx_staps,   footprint = reproject_to_healpix((M_staps['M0'], hdr2D_staps),'ICRS', nside=512)

In [ ]:
#print(M1_hpx_chime.shape)
#print(M1_hpx_hbn.shape)
#print(M1_hpx_dragons.shape)
#print(M1_hpx_lbs.shape)
print(M1_hpx_staps.shape)


In [ ]:
del chime_FD
del dragons_FD
del hbn_FD
del lbs_FD
del staps_FD
gc.collect()

## Convert STAPS to Galactic

In [ ]:
r = hp.Rotator(coord=['C', 'G'])
M1_hpx_staps_gal = np.array(r.rotate_map_pixel(M1_hpx_staps))
M0_hpx_staps_gal = np.array(r.rotate_map_pixel(M0_hpx_staps))

## Making STAPS M0 and M1 maps for Becky during POSSUM busy week

In [ ]:
#print(hdr_chime)

hdr2D = hdr_chime.copy()
wcs2D = WCS(hdr2D).dropaxis(2)

print(wcs2D)


In [ ]:
M1_car_staps_gal, footprint = reproject_from_healpix((M1_hpx_staps_gal, 'galactic'),wcs2D, nested=False)
M0_car_staps_gal, footprint = reproject_from_healpix((M0_hpx_staps_gal, 'galactic'),wcs2D, nested=False)


In [ ]:
hdr2D = wcs2D.to_header()

In [ ]:
print(repr(hdr2D))

In [ ]:
fits.writeto('/srv/data/gmims/gmims-hbs/Jan2024/staps_M1_AO_Jun3.fits',M1_car_staps_gal,hdr2D,overwrite=False)
fits.writeto('/srv/data/gmims/gmims-hbs/Jan2024/staps_M0_AO_Jun3.fits',M0_car_staps_gal,hdr2D,overwrite=False)


In [ ]:
test_hdu = fits.open('/srv/data/gmims/gmims-hbs/Jan2024/staps_M0_AO_Jun3.fits')

plt.imshow(test_hdu[0].data,vmin=0,vmax=50,cmap='cubehelix',origin='lower')

## Make masks for Galactic plane

In [ ]:
pix_indices = np.arange(hp.nside2npix(512))

lon, lat = hp.pix2ang(512, pix_indices, lonlat=True, nest=False)

print(lon)
print(lat)

chime_mask   = np.where((lon < 65) & (np.abs(lat) < 5))
dragons_mask = np.where((lon < 60) & (np.abs(lat) < 8))
hbn_mask     = np.concatenate([np.where((lon < 60) & (np.abs(lat) < 2))[0],np.where((lon > 260) & (np.abs(lat) < 2))[0]])
lbs_mask     = np.concatenate([np.where((lon < 60) & (np.abs(lat) < 8))[0],np.where((lon > 260) & (np.abs(lat) < 8))[0]])
staps_mask   = np.concatenate([np.where((lon < 60) & (np.abs(lat) < 3))[0],np.where((lon > 270) & (np.abs(lat) < 3))[0]])

## Make all plots

In [ ]:
maps   = [M1_hpx_chime,
          M1_hpx_dragons,
          M1_hpx_hbn,
          M1_hpx_lbs,
          M1_hpx_staps_gal]

titles = ['CHIME M1 586 - 729 MHz',
          'DRAGONS M1 500 - 1030 MHz', 
          'GMIMS-HBN M1 1300 - 1750 MHz',
          'GMIMS-LBS M1 300 - 480 MHz',
          'STAPS M1 1330 - 1770 MHz']

units = [r'rad m$^{-2}$',
         r'rad m$^{-2}$',
         r'rad m$^{-2}$',
         r'rad m$^{-2}$',
         r'rad m$^{-2}$']

outfiles = ['M1_chime','M1_dragons','M1_hbn','M1_lbs','M1_staps']

vmaxs = [20,20,50,10,50]

masks = [chime_mask,chime_mask,chime_mask,chime_mask,chime_mask]

do_mask = False
fs = 14
for i in range(0,5):

    map = copy.deepcopy(maps[i])
    map[map==0] = np.nan
    if do_mask:
        map[masks[i]] = np.nan

    hp.newvisufunc.projview(map,cmap='RdBu_r',min=-vmaxs[i],max=vmaxs[i],rot=(0,0),
                            xsize=2000,cbar=True,title=titles[i],unit=units[i],
                            projection_type='mollweide',fontsize={'cbar_label':fs,'cbar_tick_label':fs,'title':fs+2},
                            override_plot_properties={'cbar_shrink':0.4})

    plt.savefig('/home/aordog/GMIMS_PLOTS/all_gmims_May2024/'+outfiles[i]+'.png',dpi=200,transparent=True)

In [ ]:
maps   = [M0_hpx_chime,
          M0_hpx_dragons,
          M0_hpx_hbn,
          M0_hpx_lbs,
          M0_hpx_staps_gal]

titles = ['CHIME M0 586 - 729 MHz',
          'DRAGONS M0 500 - 1030 MHz', 
          'GMIMS-HBN M0 1300 - 1750 MHz',
          'GMIMS-LBS M0 300 - 480 MHz',
          'STAPS M0 1330 - 1770 MHz']

units = [r'Jy beam$^{-1}$ RMSF$^{-1}$ rad m$^{-2}$',
         r'Jy beam$^{-1}$ RMSF$^{-1}$ rad m$^{-2}$',
         r'K RMSF$^{-1}$ rad m$^{-2}$',
         r'K RMSF$^{-1}$ rad m$^{-2}$',
         r'K RMSF$^{-1}$ rad m$^{-2}$']

outfiles = ['M0_chime','M0_dragons','M0_hbn','M0_lbs','M0_staps']

vmaxs = [40,200,80,15,30]

masks = [chime_mask,dragons_mask,hbn_mask,lbs_mask,staps_mask]

do_mask = True
fs = 14
for i in range(0,5):
    map = copy.deepcopy(maps[i])
    map[map==0] = np.nan
    #map[map<=0.05] = np.nan
    
    if do_mask:
        map[masks[i]] = np.nan
        
    hp.newvisufunc.projview(map,cmap='cubehelix',min=0,max=vmaxs[i],rot=(0,0),
                            xsize=2000,cbar=True,title=titles[i],unit=units[i],
                            projection_type='mollweide',fontsize={'cbar_label':fs,'cbar_tick_label':fs,'title':fs+2},
                            override_plot_properties={'cbar_shrink':0.4})
    plt.savefig('/home/aordog/GMIMS_PLOTS/all_gmims_May2024/'+outfiles[i]+'.png',dpi=200,transparent=True)